## Script S4

If want to save markdown file into local, usee `S4` as a crawler.

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from neo4j import GraphDatabase, basic_auth
from neo4j.exceptions import Neo4jError
import csv
import json
import time
import os
import re
chrome_options= webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome('C:\Program Files\Google\Chrome\Application\chromedriver',options=chrome_options)
DATABASE_USERNAME="neo4j"
DATABASE_PASSWORD="spade-discounts-switch"
DATABASE_URL="bolt://localhost:7687"
DATA_FILE_PATH = 'dev-to-articles.csv'
BASE_URL = 'http://127.0.0.1:4000/'
db_driver = GraphDatabase.driver(DATABASE_URL, auth=basic_auth(DATABASE_USERNAME, str(DATABASE_PASSWORD)))

def cprint(content,module='DEBUG',*args):
    if args:
        print('\033[1;32;43m ['+module+'] \033[0m '+ content + '\033[1;35m' +str(args) +' \033[0m' + time.strftime(" |%Y-%m-%d %H:%M:%S|", time.localtime()) )
    else:
        print('\033[1;32;43m ['+module+'] \033[0m '+ content + time.strftime(" |%Y-%m-%d %H:%M:%S|", time.localtime()))
def file_write_log(path,info):
    with open(path, mode='a+',encoding="utf-8") as file_a:
        file_a.write( str(info) + time.strftime(" |%Y-%m-%d %H:%M:%S|", time.localtime()) + '\n')
    file_a.close()
FOLDER_PATH = '../web/_posts/'
test_url = 'https://dev.to/cruip/50-free-tools-and-resources-to-create-awesome-user-interfaces-1c1b'
DATA_FILE_PATH = 'dev-to-articles.csv'

In [4]:

def db_update_article_jekyll_path(session,id,value):
    '''
    Add a new value for article node with id
    Args:
        session: db session,driver.session()
        id: article id, in csv file.
        value: value of that key : reading_time
    '''
    def _cypher(tx,key,value):
        return list(tx.run(
        '''
        MATCH (n:Article { id: $id })
        SET n.jekyll_path = $value
        RETURN n
        '''
        ))
    result = session.write_transaction(_cypher,key,value)
    return result


def get_all_article(DATA_FILE_PATH):
    '''
    read csv file and collect all data
    Args: csv file path
    Return: list of article_prop
    '''
    article_props = []
    if os.path.exists(DATA_FILE_PATH):
        if not os.path.getsize(DATA_FILE_PATH):
            cprint(DATA_FILE_PATH +'is empty')
        else:
            with open(DATA_FILE_PATH, mode='r',encoding="utf-8") as data_file_r:
                csv_reader = csv.DictReader(data_file_r)
                line_count = 0
                
                for row in csv_reader:
                    if line_count == 0:
                        cprint(f'Processing CSV header {", ".join(row)}','CSV')
                        line_count += 1
                    article_prop = {
                            'id': row['id'],
                            'title': row['title'],
                            'url': row['url'],
                            'main_image_url': row['main_image_url'],
                            'reading_time': row['reading_time'],
                            'tag_names': row['tag_names'],
                            'published_at': row['published_at'],
                            'source_site':'dev.to',
                            'author_name':row['author_name'],
                            'count':row['public_reactions_count']
                    }
                    if article_prop in article_props:
                        continue
                    else:
                        article_props.append(article_prop)
                    line_count += 1
                cprint(f'File processed successfully with {line_count-1} ids.','CSV')
            data_file_r.close()

    else:
        cprint(DATA_FILE_PATH +' does not exist')
    return article_props

def save_file(article_prop):
    #Send Request
    cprint('[1]Sending request to ','Selenium',article_prop['url'])
    driver.get(article_prop['url'])
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    #send request end
    try:
        contents = soup.find("div", {"id":"article-body"}).contents
    except:
        cprint('Request failed, post might be removed','Selenium')
        return False
            
    
    #Init Yaml
    title = article_prop['title'].replace('"',"'")
    date = article_prop['published_at'][:10].replace('"',"'")
    author = article_prop['author_name'].replace('"',"'")
    id = article_prop['id']
    main_image_url =  article_prop['main_image_url']
    reading_time = article_prop['reading_time']
    yaml = ['---\n',
            'layout: post\n',
            f'id: {id}\n',
            f'title: "{title}"\n',
            f'author: "{author}"\n',
            f'date: {date}\n',
            'toc: false\n',
            f'main_image_url: "{main_image_url}"\n',
            f'reading_time: "{reading_time}"\n',
            'tags:\n'
           ]
    for tag in soup.find_all("a",{"class":"crayons-tag"}):
        yaml.append('    - '+tag.text[1:]+'\n')
    yaml.append('---\n')
    #init yaml end
    

    #save file
    
    #2020-10-22-494489.md
    filename =  article_prop['published_at'][:10]+'-'+article_prop['id']+'.md'
    #http://127.0.0.1:4000/blog/494489/
    with open(FOLDER_PATH+filename, mode='w',encoding="utf-8") as file_w:
        #write yaml
        file_w.writelines(yaml)
        #write body
        for i in contents:
            file_w.write(str(i))
        cprint('[2]Write file ssuccessfully ','FILE',FOLDER_PATH+filename)
        cprint('[3]Post should be at:','Jekyll',BASE_URL+article_prop['id']+'/')
    file_w.close()
    #save file end
    return True
    

In [5]:
#save_markdown_file(get_article_content(driver,test_url),FOLDER_PATH)
article_props = get_all_article(DATA_FILE_PATH)

 [CSV]  Processing CSV header id, title, url, main_image_url, reading_time, author_name, author_username, author_id, published_at, tag_names, keywords_for_search, comments_count, public_reactions_count, highlight |2021-05-10 13:05:55|
 [CSV]  File processed successfully with 2375 ids. |2021-05-10 13:05:55|


In [7]:
# save_file(article_props[0])

 [Selenium]  [1]Sending request to ('https://dev.to//techworld_with_nana/full-docker-course-free-4hl3',)  |2021-05-09 21:37:46|
 [FILE]  Write file ssuccessfully ('../web/_posts/2020-10-22-494489.md',)  |2021-05-09 21:37:54|
 [Jekyll]  Post should be at:('http://127.0.0.1:4000/494489/',)  |2021-05-09 21:37:54|


In [7]:
counter = 1986
for article_prop in article_props[1986:2375]:
    try:
        res = save_file(article_prop)
    except:
        time.sleep(1)
        cprint('Save file failed, filename might be invalid','FILE')
        file_write_log('markdown-error-log-v1.txt',article_prop['id'])
    if res:
        cprint(f'{counter+1} / {len(article_props)} done')
        counter+=1
    else:
        file_write_log('markdown-error-log-v1.txt',article_prop['id'])

 [Selenium]  [1]Sending request to ('https://dev.to//palaklive/7-hacks-to-optimize-your-react-app-27e6',)  |2021-05-10 16:47:33|
 [FILE]  [2]Write file ssuccessfully ('../web/_posts/2021-01-10-566868.md',)  |2021-05-10 16:47:40|
 [Jekyll]  [3]Post should be at:('http://127.0.0.1:4000/566868/',)  |2021-05-10 16:47:40|
 [DEBUG]  1987 / 2375 done |2021-05-10 16:47:40|
 [Selenium]  [1]Sending request to ('https://dev.to//martinsos/wasp-language-for-developing-full-stack-javascript-web-apps-with-no-boilerplate-3nc8',)  |2021-05-10 16:47:40|
 [FILE]  [2]Write file ssuccessfully ('../web/_posts/2021-03-04-624892.md',)  |2021-05-10 16:47:45|
 [Jekyll]  [3]Post should be at:('http://127.0.0.1:4000/624892/',)  |2021-05-10 16:47:45|
 [DEBUG]  1988 / 2375 done |2021-05-10 16:47:45|
 [Selenium]  [1]Sending request to ('https://dev.to//daiyanze/vue-3-new-features-summary-2cie',)  |2021-05-10 16:47:45|
 [FILE]  [2]Write file ssuccessfully ('../web/_posts/2020-08-11-424599.md',)  |2021-05-10 16:47:50|